# Парсинг
Выполнила: Остапчук Таисия Сергеевна, 121-ПМо


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [11]:
pages = []  # Список для хранения всех страниц с квартирами
for i in range(1, 18):
    flats_page = BeautifulSoup(requests.get(
        'http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?p=' + str(i)
    ).text, 'lxml')  # Нагло забираем страничку
    pages.append(flats_page)  # И добавляем её в списочек

In [13]:
all_links_list = []  # Список для ссылок на конкретные квартиры
all_ids_list = []  # Список для id квартир
for page in pages:  # Проходимся по страничкам
    table = page.find('div', {'class': 'wrapper'}).find('table', {'id':'tbl_10043'})  # Читаем таблицу с квартирами с нашей страницы
    for tr in table.find_all('tr'):  # Проходимся по строчкам таблицы
      if len(tr) != 13:  # Находим строчки с нужной информацией
          continue
      id = int(tr.find_all('td', recursive=False)[0].input['value'])  # Считываем id из input
      link = tr.find_all('td', recursive=False)[4].a['href']  # Считываем ссылочку из address
      all_links_list.append(link)  # Заносим всё, что нашли в наши списочки
      all_ids_list.append(id)

In [14]:
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь',
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
data = pd.DataFrame(index=all_ids_list, columns=columns)  # Создаем DataFrame
data['Ссылка'] = all_links_list  # Сразу заполняем столбик ссылок
data

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23812769,/realty/prodazha-kvartir/prodam-kvartiru-23812...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812768,/realty/prodazha-kvartir/prodam-kvartiru-23812...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812767,/realty/prodazha-kvartir/prodam-kvartiru-23812...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812766,/realty/prodazha-kvartir/prodam-kvartiru-23812...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812765,/realty/prodazha-kvartir/prodam-kvartiru-23812...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16262155,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15406262,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16022577,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15761115,/realty/prodazha-kvartir/prodam-kvartiru-15761...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for i in range(len(all_ids_list)):  # Проходимся по всем страницам с конкретными кавртирами
  flat_page = BeautifulSoup(requests.get(
        'http://magnitogorsk-citystar.ru' + all_links_list[i]
    ).text, 'lxml')
  div = flat_page.find('div', 'adv-main-data')  # Находим контейнер с необходимой инофрмацией
  key_list = [key.get_text() for key in div.find_all('td', 'field-title')]  # Список с ключами для словаря с информацией о квартире (заголовки)
  value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]  # Список уже с самой информацией (цена, площадь и т.д.)
  data_dict = dict(zip(key_list, value_list))  # Склеиваем
  if div.find('td', 'note'):  # Отдельно проверяем наличия описания квартиры
        data_dict['Описание'] = div.find('td', 'note').get_text()  # И добавляем, если нашли
  data.loc[all_ids_list[i], 'Цена':] = data_dict  # Обновляем строку с квартирой в нашей таблице

In [16]:
data.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23812769,/realty/prodazha-kvartir/prodam-kvartiru-23812...,2080000р. (63609р./м2)Подать заявку на ипотеку,NaN,"Ленина пр-т, 212б",1/1,NaN,NaN,"общая 32,7 м2, жилая 30 м2, кухни 0 м2",NaN,NaN,id:47638. \nПредставьте себе уединённый уголок...
23812768,/realty/prodazha-kvartir/prodam-kvartiru-23812...,2080000р. (65000р./м2)Подать заявку на ипотеку,NaN,"Ленина пр-т, 214",1/2,NaN,NaN,"общая 32 м2, жилая 20 м2, кухни 12 м2",NaN,Однокомнатная,id:47637. \nУстали от городского шума и суеты?...
23812767,/realty/prodazha-kvartir/prodam-kvartiru-23812...,1560000р. (65000р./м2)Подать заявку на ипотеку,NaN,NaN,1/2,NaN,NaN,"общая 24 м2, жилая 18 м2, кухни 5 м2",NaN,Однокомнатная,id:47636. \nУстали от городского шума и суеты?...
23812766,/realty/prodazha-kvartir/prodam-kvartiru-23812...,1250000р. (30488р./м2)Подать заявку на ипотеку,NaN,"Садовая, 18",2/2,NaN,NaN,"общая 41 м2, жилая 20 м2, кухни 7 м2",NaN,Однокомнатная,id:47633. \nПродам уютную 1-комнатную квартиру...
23812765,/realty/prodazha-kvartir/prodam-kvartiru-23812...,2770000р. (63532р./м2)Подать заявку на ипотеку,NaN,"Советская, 145/1",1/5,NaN,NaN,"общая 43,6 м2, жилая 30 м2, кухни 6 м2",NaN,Двухкомнатная,id:47626. \nПредставляем вашему вниманию уютну...


In [17]:
data.to_csv('magnitogorsk_parsing.csv')
data.to_excel('magnitogorsk_parsing.xlsx')